## Recreating digest2 output for all the XML files 
We want to recreate the digest2 output for all the XML files from the NEO Surveyor mission. The digest2 score needs to contain all the available values. 

#### Setting up files and directories 

In [6]:
# Import
import shutil
import subprocess
import time
import os

import xmltodict
import pandas as pd


### Initial data

In [3]:
# Directory containing the XML files
xml_dir = "NEOSurveyordata-20241021/"
xml_files = [
    "2024-10-21T16_54_54.398_000000R8.xml"
]

# Off-ecliptic and on-ecliptic XML files
off_ecliptic_file = xml_dir+"off_ecl_tracklet2desig3.dat"
on_ecliptic_file = xml_dir+"on_ecl_tracklet2desig3.dat"

# Digest2 executable
digest2_exec = "digest2/digest2/digest2"

### ROUTINES

In [ ]:
#Analyze each XML file
def analyze_xml(xml_file):
    """ Analyze the content of the XML file. """
    with open(xml_file, 'r') as file:
        xml_string = file.read()

    # Convert the XML string to a Python dictionary
    xml_dict = xmltodict.parse(xml_string)
        
    # Convert the dictionary to a DataFrame
    xml_df = pd.DataFrame(xml_dict['ades']['obsBlock']['obsData']['optical'])
    
    # Count the number of observations
    print(f'{xml_df.shape[0]}  observations in  {xml_file}')
    # Count the number of unique trksubs
    print(f'{xml_df["trkSub"].nunique()}  unique trksubs in  {xml_file}')


2522  observations in  NEOSurveyordata-20241021/2024-10-21T16_54_54.398_000000R8.xml
558  unique trksubs in  NEOSurveyordata-20241021/2024-10-21T16_54_54.398_000000R8.xml


In [11]:
# Run digest2 code on single file
def run_digest2(file_path):
    # Check if the file exists
    if not os.path.isfile(file_path):
        print(f"File {file_path} does not exist.")
        return False
    print(f"Processing: {file_path}")

    start_time = time.time() # starting a timer
    
    try:
        result = subprocess.run(
        f"{digest2_exec} -c MPC.config {file_path}",
        capture_output=True, text=True, check=True, shell=True
        )
        if result.stderr:
            print(f"Error: {result.stderr}")

    except subprocess.CalledProcessError as e:
        print(f"Error processing {file_path}: {e}")
        print(f"Command output: {e.output}")
        return False

    end_time = time.time() # ending the timer
    elapsed_time = end_time - start_time

    print(f"Time taken for {file_path}: {elapsed_time:.4f} seconds\n")
    
    return result

In [5]:
# Match trksubs with designations from NEO Surveyor 
def match_desig_trksub(offecliptic_file:str, onecliptic_file:str) -> dict:
    """ Create a dictionary that matches the trksubs to the designations given by the NEO Surveyor Team """

    file_list = [offecliptic_file, onecliptic_file]
    # Empty dictionary
    trksub_desig = {}
    
    # Count the number of NEAs and not NEAs in the file
    nnea = 0
    nnonnea = 0
    for f in file_list:
        # Check if the file exists
        if not os.path.isfile(f):
            print(f"File {f} not found.")
            return None
        # Open the file 
        with open(f, 'r', encoding='utf-8') as file:
            lines = file.readlines()
            for line in lines:
                # Create a dictionary to store the matches
                sp = line.split()
                trksub = sp[0]
                desig = sp[1]
                # NEA case
                if desig[3] == "0":
                    NEA = '0'
                    nnea += 1
                else:
                    # MBA case
                    NEA = '1'
                    nnonnea += 1
                trksub_desig[trksub] = [desig, NEA]
                
    # Print the number of NEAs and not NEAs
    print(f"Total number of NEAs in the off-ecliptic and on-ecliptic data: {nnea}")
    print(f"Total number of non-NEAs in the off-ecliptic and on-ecliptic data: {nnonnea}")

    return trksub_desig
        

In [82]:
# Create the output needed by the machine learning code 
def create_output_format(xml_file: str, digest2_result:list, trksub_desig : dict) -> None:
    """ 
    Create the output format needed by the machine learning code.
    """
    # Create output file 
    filtering_output = open(xml_file.replace(".xml",".digest2_filter"), "w", encoding='utf-8')
    ml_output = open(xml_file.replace(".xml",".digest2_ml"), "w", encoding='utf-8')
    output = open(xml_file.replace(".xml",".digest2"), "w", encoding='utf-8')
    # Write header
    output.write("trksub,Int1,Int2,Neo1,Neo2,MC1,MC2,Hun1,Hun2,Pho1,Pho2,MB1_1,MB1_2,Pal1,Pal2,Han1,Han2,MB2_1,MB2_2,MB3_1,MB3_2,Hil1,Hil2,JTr1,JTr2,JFC1,JFC2\n")
    # If filtering is true, add the NEA class to the header and call it "class"
    filtering_output.write("trksub,Int1,Int2,Neo1,Neo2,MC1,MC2,Hun1,Hun2,Pho1,Pho2,MB1_1,MB1_2,Pal1,Pal2,Han1,Han2,MB2_1,MB2_2,MB3_1,MB3_2,Hil1,Hil2,JTr1,JTr2,JFC1,JFC2,class\n")
    # If machine learning is true, add the NEA class to the header and call it "orbtype"
    ml_output.write("trksub,Int1,Int2,Neo1,Neo2,MC1,MC2,Hun1,Hun2,Pho1,Pho2,MB1_1,MB1_2,Pal1,Pal2,Han1,Han2,MB2_1,MB2_2,MB3_1,MB3_2,Hil1,Hil2,JTr1,JTr2,JFC1,JFC2,orbtype\n")
        
    # Count the number of trksubs processed
    ntrksub = 0
    # Count the number of matched trksubs
    nmatched = 0
    nnea = 0
    nmba = 0
    
    # Read digest2 results
    for line in digest2_result.splitlines()[2:]:
        # Split line into columns
        sp = line.split()

        # Increment the number of trksubs processed
        ntrksub += 1
        
        # Always write the output file for digest2
        output.write(','.join(sp) + "\n")
        
        # Find if the object is an NEA or not
        if sp[0] not in trksub_desig:
            # If the object is not in the dictionary
            continue
        
        # Increment the number of matched trksubs
        nmatched += 1
        NEAclass = trksub_desig[sp[0]][1]
        if NEAclass == '0':
            nnea += 1
        else:
            nmba += 1
        output_string = ','.join(sp) + ',' + NEAclass + "\n"
        # Write to filtering output file
        filtering_output.write(output_string)
        # Write to machine learning output file
        ml_output.write(output_string)
        
    # Close output file
    output.close()
    filtering_output.close()
    ml_output.close()
    
    print(f"Processed {ntrksub} trksubs, matched {nmatched} trksubs in {xml_file}.")
    print(f"Found {nnea} NEAs and {nmba} MBAs in {xml_file}.")

    print(f"Output file {xml_file.replace('.xml','.digest2')} created.")
    print(f"Output file {xml_file.replace('.xml','.digest2_filter')} created.")
    print(f"Output file {xml_file.replace('.xml','.digest2_ml')} created.")


In [64]:
def create_digest2_output(xml_dir: str, xml_files: list, trksub_desig: dict) -> None:
    
    # Run digest2 on single files
    for xml_file in xml_files:
        
        # Analyze the XML file
        print(" ------ Analyze XML file ------ ")
        analyze_xml(os.path.join(xml_dir, xml_file))

        # Run digest2 on the XML file
        print(" ------ Run digest2 on XML file ------ ")
        digest2_result = run_digest2(os.path.join(xml_dir, xml_file))
        if not digest2_result:
            print(f"Digest2 failed for {xml_file}. Skipping further processing.")
            continue
        
        if not trksub_desig:
            print("Failed to match trksubs with designations. Skipping further processing.")
            continue
        
        # Process the results
        print(" ------ Create output format ------ ")
        create_output_format(xml_file, digest2_result.stdout, trksub_desig)

In [ ]:
# Analyze output from digest2
def analyze_digest2_output(digest2_file: str, digest2_thresh : int = 65) -> None:
    """ Analyze the output from digest2. """

    output_df = pd.read_csv(digest2_file)
    
    # Count the number of NEAs and non-NEAs
    nea_count = output_df[output_df['Neo2'] >= digest2_thresh].shape[0]
    nonnea_count = output_df[output_df['Neo2'] < digest2_thresh].shape[0]

    print(f"Number of NEAs in {digest2_file}: {nea_count}")
    print(f"Number of non-NEAs in {digest2_file}: {nonnea_count}")
    
# Analyze missing NEAs from digest2
def analyze_missing_NEAs(digest2_file: str, digest2_thresh : int = 65) -> None:
    """ Analyze the output from digest2. """

    output_df = pd.read_csv(digest2_file)
    
    # Check NEAs in NON_NEA df
    notnea_df = output_df[output_df['Neo2'] < digest2_thresh]
    misidentified_nea = notnea_df[notnea_df['class'] == 0]
    print(f"Number of misidentified NEAs in {digest2_file}: {misidentified_nea.shape[0]}")
    #print(misidentified_nea)
    
    # Check for non-NEAs in NEA df
    nea_df = output_df[output_df['Neo2'] >= digest2_thresh]
    misidentified_nonnea = nea_df[nea_df['class'] == 1]
    print(f"Number of misidentified non-NEAs in {digest2_file}: {misidentified_nonnea.shape[0]}")
    #print(misidentified_nonnea)


In [16]:
def analyze_optimal_thresholds(optimal_thresh_file: str, digest2_file: str) -> None:
    """ Analyze the optimal thresholds from digest2. """

    # The file only contains the trksubs that are not NEAs
    thresh_df = pd.read_csv(optimal_thresh_file, header=None, names=['trksub'])
    print(f"Number of trksubs in {optimal_thresh_file}: {thresh_df.shape[0]}")
    digest2_df = pd.read_csv(digest2_file)
    
    # Count the number of NEAs and non-NEAs
    neas = digest2_df[digest2_df["class"] == 0]
    # We want to check if there are NEAs in the optimal thresholds file
    nea_count = neas[neas['trksub'].isin(thresh_df['trksub'])].shape[0]
    print(f"Number of NEAs in {optimal_thresh_file}: {nea_count}")
    #print(neas[neas['trksub'].isin(thresh_df['trksub'])])


### Main

In [6]:
""" Main execution flow 
1. Create a dictionary that matches the trksubs to the designations given by the NEO Surveyor Team
2. Analyze each XML file
3. Run digest2 on each XML file
4. Create the output format needed by the filtering and the machine learning code
"""

# Match trksubs with designations
print(" ------ Match trksubs with designations ------ ")
trksub_desig = match_desig_trksub(off_ecliptic_file, on_ecliptic_file)




 ------ Match trksubs with designations ------ 
Total number of NEAs in the off-ecliptic and on-ecliptic data: 1280
Total number of non-NEAs in the off-ecliptic and on-ecliptic data: 550120


In [ ]:
print(" ------ Create digest2 output format for each XML file ------ ")
create_digest2_output(xml_dir, xml_files, trksub_desig)

In [15]:
""" Main execution flow 
5. Analyze the output files 
"""
# Analyze the output files
for xml_file in xml_files:
    print(" ------ Analyze output files ------ ")
    #analyze_digest2_output(xml_file.replace(".xml", ".digest2"),digest2_thresh=90)
    analyze_digest2_output(xml_file.replace(".xml", ".digest2_filter"), digest2_thresh=100)

 ------ Analyze output files ------ 
Number of NEAs in 2024-10-21T16_54_54.398_000000R8.digest2_filter: 36
Number of non-NEAs in 2024-10-21T16_54_54.398_000000R8.digest2_filter: 418


#### Analyzing the digest2 score for the missing NEAs
We noticed that if we set digest2=100, we are then missing some NEAs (between 60% and 74% of objects are actually correctly identified) 

In [32]:
# Analyze the output files
for xml_file in xml_files:
    print(" ------ Analyze missing NEAs ------ ")
    analyze_missing_NEAs(xml_file.replace(".xml", ".digest2_filter"), digest2_thresh=100)

 ------ Analyze missing NEAs ------ 
Number of misidentified NEAs in 2024-10-21T16_54_54.398_000000R8.digest2_filter: 14
Number of misidentified non-NEAs in 2024-10-21T16_54_54.398_000000R8.digest2_filter: 1


### Optimal threshold

In [3]:
# Import code from github repository
from neocp_filter import filter_and_output_passed_entries

In [8]:
for xml in xml_files:
    print(f"Processing file: {xml}")
    input_csv = xml.replace(".xml", ".digest2_filter")
    threshold_json = f"/Users/fspoto/Work/MPC/Work_in_progress/NEOSurveyor/Fulldigest2/ML_digest2_methods/data/optimal_thresholds.json"
    output_csv = input_csv.replace(".digest2_filter", "_filter_passed.csv")

    filter_and_output_passed_entries(input_csv, threshold_json, output_csv)

Processing file: 2024-10-21T16_54_54.398_000000R8.xml


In [17]:
# Analyze optimal thresholds output
for xml in xml_files:
    optimal_thresh_file = xml.replace(".xml", "_filter_passed.csv")
    digest2_file = xml.replace(".xml", ".digest2_filter")
    print(f"Analyzing optimal thresholds for {optimal_thresh_file} and {digest2_file}")
    analyze_optimal_thresholds(optimal_thresh_file, digest2_file) 

Analyzing optimal thresholds for 2024-10-21T16_54_54.398_000000R8_filter_passed.csv and 2024-10-21T16_54_54.398_000000R8.digest2_filter
Number of trksubs in 2024-10-21T16_54_54.398_000000R8_filter_passed.csv: 312
Number of NEAs in 2024-10-21T16_54_54.398_000000R8_filter_passed.csv: 1


### Running the machine learning code

In [7]:
ml_python = "ML_digest2_methods/src/testing_pipeline.py"
model_dir = "ML_digest2_methods/models/"
for xml in xml_files:
    print(f"Processing file: {xml}")
    input_csv = xml.replace(".xml", ".digest2_ml")
    # Run the machine learning code
    subprocess.run(
        f"python3 {ml_python} --model_dir {model_dir} --test_data {input_csv} --save_results",
        shell=True, check=True
    )
    # Copy the results to the current directory
    shutil.copyfile(f"ML_digest2_methods/models/GBM_predictions.csv", f"{xml.replace('.xml', '_GBM_predictions.csv')}")
    shutil.copyfile(f"ML_digest2_methods/models/confusion_matrices.png", f"{xml.replace('.xml', '_confusion_matrices.png')}")

Processing file: 2024-10-21T16_54_54.398_000000R8.xml


2025-07-03 10:31:10.512726: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Loading test data...

=== Evaluating GBM ===
Shape of input to GBM predict_proba: (454, 26)
Accuracy: 0.9692

Classification Report:
              precision    recall  f1-score   support

           0       0.87      0.84      0.85        49
           1       0.98      0.99      0.98       405

    accuracy                           0.97       454
   macro avg       0.93      0.91      0.92       454
weighted avg       0.97      0.97      0.97       454


Confusion Matrix:
[[ 41   8]
 [  6 399]]
Results saved to ML_digest2_methods/models/GBM_predictions.csv

=== Evaluating RF ===
Accuracy: 0.9670

Classification Report:
              precision    recall  f1-score   support

           0       0.87      0.82      0.84        49
           1       0.98      0.99      0.98       405

    accuracy                           0.97       454
   macro avg       0.92      0.90      0.91       454
weighted avg       0.97      0.97      0.97       454


Confusion Matrix:
[[ 40   9]
 [  6 399]]
Re

/Users/fspoto/anaconda3/envs/NEOSurveyor/lib/python3.11/pickle.py:1718: UserWarning: [10:31:15] WARNING: /Users/runner/work/xgboost/xgboost/src/gbm/../common/error_msg.h:82: If you are loading a serialized model (like pickle in Python, RDS in R) or
configuration generated by an older version of XGBoost, please export the model by calling
`Booster.save_model` from that version first, then load it back in current version. See:

    https://xgboost.readthedocs.io/en/stable/tutorials/saving_model.html

for more details about differences between saving model and serializing.

  setstate(state)
/Users/fspoto/anaconda3/envs/NEOSurveyor/lib/python3.11/site-packages/sklearn/base.py:440: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.5.1 when using version 1.7.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-li

15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step  
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
Accuracy: 0.9383

Classification Report:
              precision    recall  f1-score   support

           0       0.66      0.88      0.75        49
           1       0.98      0.95      0.96       405

    accuracy                           0.94       454
   macro avg       0.82      0.91      0.86       454
weighted avg       0.95      0.94      0.94       454


Confusion Matrix:
[[ 43   6]
 [ 22 383]]
Results saved to ML_digest2_methods/models/NN_predictions.csv

=== Model Comparison ===
Model  Accuracy
  GBM  0.969163
   RF  0.966960
   NN  0.938326

Detailed results saved to ML_digest2_methods/models/model_evaluation_results.txt
